# test-api.ipynb

Test API and helper functions

In [1]:
import os, sys
from typing import Optional, List, Dict
sys.path.append(os.path.join(".."))
import hjson
import docstring_parser
import inspect
from llms_wrapper.llms import LLMS
from llms_wrapper.config import update_llm_config

In [2]:
config = dict(
    llms=[
        # OpenAI
        # https://platform.openai.com/docs/models
        dict(llm="openai/gpt-4o"),
        dict(llm="openai/gpt-4o-mini"),
        # dict(llm="openai/o1"),        # restricted
        # dict(llm="openai/o1-mini"),   # restricted
        # Google Gemini
        # https://ai.google.dev/gemini-api/docs/models/gemini
        dict(llm="gemini/gemini-2.0-flash-exp"),
        dict(llm="gemini/gemini-1.5-flash"),
        dict(llm="gemini/gemini-1.5-pro"),
        # Anthropic
        # https://docs.anthropic.com/en/docs/about-claude/models
        dict(llm="anthropic/claude-3-5-sonnet-20240620"),
        dict(llm="anthropic/claude-3-opus-20240229"),
        # Mistral
        # https://docs.mistral.ai/getting-started/models/models_overview/
        dict(llm="mistral/mistral-large-latest"),
        # XAI
        # dict(llm="xai/grok-2"),     # not mapped by litellm yet?
        dict(llm="xai/grok-beta"),
        # Groq
        # https://console.groq.com/docs/models
        dict(llm="groq/llama3-70b-8192"),
        dict(llm="groq/llama-3.3-70b-versatile"),
        # Deepseek
        # https://api-docs.deepseek.com/quick_start/pricing
        dict(llm="deepseek/deepseek-chat"),
        dict(
            llm="gemini/somemodel",
            max_input_tokens=100000,
            cost_per_prompt_token=0.0002,
            temperature=0,
        ), 
    ],
    providers = dict(
        openai = dict(api_key_env="MY_OPENAI_API_KEY"),
        gemini = dict(api_key_env="MY_GEMINI_API_KEY"),
        anthropic = dict(api_key_env="MY_ANTHROPIC_API_KEY"),
        mistral = dict(api_key_env="MY_MISTRAL_API_KEY"),
        xai = dict(api_key_env="MY_XAI_API_KEY"),    
        groq = dict(api_key_env="MY_GROQ_API_KEY"),
        deepseek = dict(api_key_env="MY_DEEPSEEK_API_KEY"),
    )
)
_ = update_llm_config(config)

In [3]:
llms = LLMS(config)

In [4]:
llms.list_aliases()

['openai/gpt-4o',
 'openai/gpt-4o-mini',
 'gemini/gemini-2.0-flash-exp',
 'gemini/gemini-1.5-flash',
 'gemini/gemini-1.5-pro',
 'anthropic/claude-3-5-sonnet-20240620',
 'anthropic/claude-3-opus-20240229',
 'mistral/mistral-large-latest',
 'xai/grok-beta',
 'groq/llama3-70b-8192',
 'groq/llama-3.3-70b-versatile',
 'deepseek/deepseek-chat',
 'gemini/somemodel']

In [5]:
llms.known_models()

TypeError: 'bool' object is not iterable

In [5]:
llms.cost_per_token("openai/gpt-4o")

(1e-05, 2.5e-06)

In [6]:
llms.max_input_tokens("openai/gpt-4o")

16384

In [7]:
llms.max_output_tokens("openai/gpt-4o")

16384

In [8]:
llms["gemini/somemodel"].config

{'llm': 'gemini/somemodel',
 'max_input_tokens': 100000,
 'cost_per_prompt_token': 0.0002,
 'temperature': 0,
 'api_key_env': 'MY_GEMINI_API_KEY',
 'alias': 'gemini/somemodel',
 '_cost': 0,
 '_elapsed_time': 0}

In [9]:
llms.cost_per_token("gemini/gemini-1.5-flash")

(3e-07, 7.5e-08)

In [10]:
llms.cost_per_token("gemini/somemodel")

(0.0002, None)

In [11]:
llms.max_input_tokens("gemini/somemodel")

100000

In [12]:
llms.max_output_tokens("gemini/somemodel")

In [14]:
msg1 = llms.make_messages("What is a monoid?")

[{'content': 'What is a monoid?', 'role': 'user'}]

In [15]:
msg2 = llms.make_messages("What is a monoid? Return a JSON dict that has the single key 'answer' that contains your answer.")

In [18]:
llms.query(llmalias="openai/gpt-4o", messages=msg2, response_format=dict(type="json_object"))

{'elapsed_time': 4.468285083770752,
 'answer': '{"answer": "A monoid is an algebraic structure with a single associative binary operation and an identity element. Formally, a set M is a monoid if it is equipped with a binary operation \\\\(\\\\ast: M \\\\times M \\\\to M\\\\) such that: \\n1. Associativity: For all a, b, c in M, \\\\((a \\\\ast b) \\\\ast c = a \\\\ast (b \\\\ast c)\\\\).\\n2. Identity element: There exists an element e in M such that for all elements a in M, \\\\(e \\\\ast a = a \\\\ast e = a\\\\). \\nMonoids are foundational structures in abstract algebra and have applications in computer science and other fields."}',
 'error': '',
 'ok': True}

In [13]:
def func1(a: str, b: int, c: int = 1, d: Optional[List[Dict]] = None) -> str: 
    """
    This is the short description.

    Here we may have a longer description. This one can go over many lines

    :param str a: this is parameter a
    :param b: this is parameter b
    :type b: int
    :param c: some parameter c    
    :param d: some parameter d
    :return: what it returns
    :rtype: str
    """
    return "x"


In [14]:
llms.make_tooling(func1)

[{'type': 'function',
  'function': {'name': 'func1',
   'description': 'Here we may have a longer description. This one can go over many lines',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'str', 'description': 'this is parameter a'},
     'b': {'type': 'int', 'description': 'this is parameter b'},
     'c': {'type': 'int', 'description': 'some parameter c'},
     'd': {'type': 'Optional', 'description': 'some parameter d'}},
    'required': ['a', 'b']}}}]

In [ ]:
doc = docstring_parser.parse(func1.__doc__)

In [ ]:
doc.params[0]

In [ ]:
len(doc.params)

## Test Retries

In [16]:
config2 = dict(
    llms = [
        dict(llm="ollama/llama3", api_url="http://localhost:11434", num_retries=3)
    ]
)
config2

{'llms': [{'llm': 'ollama/llama3',
   'api_url': 'http://localhost:11434',
   'num_retries': 3}]}

In [18]:
llms2 = LLMS(config2, use_phoenix=("http://0.0.0.0:6006/v1/traces", "llms_wrapper_test"))
# llms2 = LLMS(config2)
llms2["ollama/llama3"].config

Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: llms_wrapper_test
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://0.0.0.0:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



{'llm': 'ollama/llama3',
 'api_url': 'http://localhost:11434',
 'num_retries': 3,
 '_cost': 0,
 '_elapsed_time': 0}

In [19]:
messages = llms2.make_messages(query="What is a monoid")

In [20]:
ret = llms2.query(
    "ollama/llama3", 
    messages=messages, 
    # return_cost=True,
    debug=True,
    num_retries=0,
)
ret

{'elapsed_time': 7.766540288925171,
 'answer': 'A fundamental question!\n\nIn abstract algebra, a monoid is an algebraic structure consisting of:\n\n1. A set M (the underlying set)\n2. An associative binary operation ∘ (called the multiplication or product) that combines two elements of M to produce another element in M.\n\nTo be more specific, a monoid M satisfies the following properties:\n\n1. **Associativity**: For all a, b, c ∈ M, it holds that (a ∘ b) ∘ c = a ∘ (b ∘ c). This means that the order in which you combine elements doesn\'t matter.\n2. **Identity element**: There exists an element e ∈ M, called the identity or neutral element, such that for all a ∈ M, a ∘ e = e ∘ a = a.\n\nIn other words, a monoid is like a set of things with a way to combine them (using the multiplication operation), where:\n\n* Combining elements in any order doesn\'t change the result.\n* There\'s a special "neutral" element that leaves anything unchanged when combined with it.\n\nMonoids are importa